## OSPI Enrollment 2023-24 Dataset

The Office of the Superintendent of Public Instruction (OSPI) maintains a repository of data that is available for public use. In my previous role as the tuition and financial aid director for a private high school, my team and I used data from OSPI to gain understanding of overall student enrollment in Washington State and to guide our strategic enrollment goals and long-term budgeting.


In [ ]:
# Import necessary libraries for assignment
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:

# Reads OSPI enrollment CSV files into a DataFrame.

def load_enrollment_data(file_path):    
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        return df
    
    except FileNotFoundError:
        print(f'File not found: {file_path}')
        return None

# Read OSPI Enrollment Data into DataFrames    
df_24 = load_enrollment_data('Report_Card_Enrollment_2023-24.csv')
df_23 = load_enrollment_data('Report_Card_Enrollment_2022-23.csv')
df_22 = load_enrollment_data('Report_Card_Enrollment_2021-22.csv')
df_21 = load_enrollment_data('Report_Card_Enrollment_2020-21.csv')
df_20 = load_enrollment_data('Report_Card_Enrollment_2019-20.csv')



In [48]:
# Function to reorder columns based on a reference DataFrame, then concatenate
def reorder_and_concat(*dataframes):
    try:
        # Standardize column names
        for df in dataframes:
            df.columns = df.columns.str.lower()  # Convert to lowercase
            df.columns = df.columns.str.replace(r"[_\-/]", " ", regex=True)  # Replace _, -, or / with a space
        
        # Use the columns from the first DataFrame as the reference
        reference_columns = dataframes[0].columns
        
        # Reorder each DataFrame to match the reference column order
        reordered_dfs = [df[reference_columns] for df in dataframes]
        
        # Concatenate the reordered DataFrames along rows
        concatenated_df = pd.concat(reordered_dfs, axis=0)
        
        print("DataFrames concatenated successfully with aligned columns.")
        return concatenated_df
    
    except Exception as e:
        print("Columns do not match. Concatenation failed.")
        return None

concat_df = reorder_and_concat(df_24, df_23, df_22, df_21, df_20)
print(concat_df) # Will print None if concatenation fails


DataFrames concatenated successfully with aligned columns.
      schoolyear organizationlevel    county  \
0        2023-24          District     Adams   
1        2023-24          District     Adams   
2        2023-24          District     Adams   
3        2023-24          District     Adams   
4        2023-24          District     Adams   
...          ...               ...       ...   
19934    2019-20             State  Multiple   
19935    2019-20             State  Multiple   
19936    2019-20             State  Multiple   
19937    2019-20             State  Multiple   
19938    2019-20             State  Multiple   

                                esdname  esdorganizationid  districtcode  \
0      Educational Service District 101            100,001         1,109   
1      Educational Service District 101            100,001         1,109   
2      Educational Service District 101            100,001         1,109   
3      Educational Service District 101            100,001  

In [49]:
# Analyze the dataset to create baseline understanding

print('Display the first 5 rows of the OSPI dataframe')
print(concat_df.head(5)) # Display the first 5 rows of the Dataframe
print("*"*75)

print('Dataframe info:')
print(concat_df.info()) # Display info about the data, including shape & column info
print("*"*75)


print('Missing data:')
print(concat_df.isnull().sum()) # Display columns with missing data


Display the first 5 rows of the OSPI dataframe
  schoolyear organizationlevel county                           esdname  \
0    2023-24          District  Adams  Educational Service District 101   
1    2023-24          District  Adams  Educational Service District 101   
2    2023-24          District  Adams  Educational Service District 101   
3    2023-24          District  Adams  Educational Service District 101   
4    2023-24          District  Adams  Educational Service District 101   

   esdorganizationid  districtcode               districtname  \
0            100,001         1,109  Washtucna School District   
1            100,001         1,109  Washtucna School District   
2            100,001         1,109  Washtucna School District   
3            100,001         1,109  Washtucna School District   
4            100,001         1,109  Washtucna School District   

   districtorganizationid  schoolcode      schoolname  ...  non foster care  \
0                 100,287       

### Business Questions for Analysis

- How is enrollment trending since 2020/post-COVID era (2020-2021, 2021-2022, 2022-2023, 2023-2024)
    - by class
- Can we see a heatmap of enrollment increases/decreases across school districts?
- Isolate Spokane area schools and do same analysis as above
    - may require school - zip code dataset to cross-reference
    - is Spokane County good enough for aggregation?


In [ ]:
# Remove rows with missing or duplicated values from dataframe

cleaned_df = concat_df.dropna()
cleaned_df = cleaned_df.drop_duplicates() 
cleaned_df

## Enrollment Trends

Using merged datasets from above, produce plot line of enrollment change from 2020 to present.

In [ ]:
enrollment_data = concat_df.pivot_table(values="All Students", index="GradeLevel", columns="SchoolYear", aggfunc="sum")
enrollment_data = enrollment_data.style.format("{:,.0f}")

enrollment_data

In [ ]:
enrollment_data = concat_df.pivot_table(values="All Students", index=["SchoolName","GradeLevel"], columns=["County","SchoolYear"], aggfunc="sum", fill_value=0)
enrollment_data = enrollment_data.style.format("{:,.0f}")

enrollment_data

In [ ]:
print('Dataframe objects:')
print(cleaned_df.columns) # Display the columns for df.drop call

<mark> this code block will drop columns but is not ready for use. drop into Code block when ready to execute <mark>


trunc_df = cleaned_df.drop([['SchoolYear', 'OrganizationLevel', 'County', 'ESDName',
       'ESDOrganizationID', 'DistrictCode', 'DistrictName',
       'DistrictOrganizationId', 'SchoolCode', 'SchoolName',
       'SchoolOrganizationID', 'CurrentSchoolType', 'GradeLevel',
       'All Students', 'Female', 'Gender X', 'Male',
       'American Indian/ Alaskan Native', 'Asian', 'Black/ African American',
       'Hispanic/ Latino of any race(s)',
       'Native Hawaiian/ Other Pacific Islander', 'Two or More Races', 'White',
       'English Language Learners', 'Foster Care', 'Highly Capable',
       'Homeless', 'Low-Income', 'Migrant', 'Military Parent', 'Mobile',
       'Section 504', 'Students with Disabilities',
       'Non-English Language Learners', 'Non-Foster Care',
       'Non-Highly Capable', 'Non-Homeless', 'Non-Low Income', 'Non Migrant',
       'Non Military Parent', 'Non Mobile', 'Non Section 504',
       'Students without Disabilities', 'DataAsOf']], axis=1)

In [ ]:
# Display statistical summary of data
print('Statistical summary of dataframe')
print(concat_df.describe())
